<a href="https://colab.research.google.com/github/yeneua/HG-MLDL/blob/main/05_3_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>05-3 트리의 앙상블</h2>

랜덤 포레스트

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [4]:
from sklearn.model_selection import cross_validate # 교차검증 함수
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트
rf = RandomForestClassifier(n_jobs = -1, random_state = 42) # 랜덤포레스트 객체할당

In [8]:
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1) # 교차검증

In [10]:
print(scores) # 5폴드 교차검증(기본)

{'fit_time': array([0.87527514, 1.08443785, 0.86284971, 0.94291329, 0.74418497]), 'score_time': array([0.20713425, 0.12287021, 0.12284541, 0.12220669, 0.10463834]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}


In [11]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [13]:
rf.fit(train_input, train_target) # 전체 훈련세트로 훈련
print(rf.feature_importances_) # 변수 중요도 출력

[0.23167441 0.50039841 0.26792718]


In [16]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_) # 교차검증점수와 비슷

0.8934000384837406


엑스트라 트리

In [18]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [21]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


그레이디언트 부스팅

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 그레이디언트 교차검증 점수 

0.8881086892152563 0.8720430147331015


In [29]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42) # 결정트리개수 - 500개, learning_rate : 학습속도제어
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [30]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


히스토그램 기반 그레이디언트 부스팅

In [46]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 히스토그램 기반 그레이디언트 부스팅 교차검증 점수

0.9321723946453317 0.8801241948619236


In [33]:
from sklearn.inspection import permutation_importance # 변수 중요도

In [48]:
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [50]:
# => 두번째 특성을 섞으면 정확도가 0.23만큼 떨어진다는 의미
# => 두번째 특성이 가장 중요하다

In [36]:
print(result)
# permutation_importance()
# importances 특성중요도
# importances_mean 특성중요도 평균
# importances_std 표준편차

{'importances_mean': array([0.08876275, 0.23438522, 0.08027708]), 'importances_std': array([0.00382333, 0.00401363, 0.00477012]), 'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
        0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
       [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
        0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
       [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
        0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]])}


In [37]:
result = permutation_importance(hgb, test_input, test_target, # 테스트 세트
                                n_repeats = 10, # 랜덤하게 섞을 횟수
                                random_state = 42, n_jobs = -1)
print(result.importances_mean) # 이 모델을 실전에 투입했을 때, 어떤 특성에 더 중점을 줄 것인지 ?

[0.05969231 0.20238462 0.049     ]


In [38]:
hgb.score(test_input, test_target) # 테스트세트에서 성능 최종적으로 확인

0.8723076923076923

In [41]:
# XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [45]:
# lightgbm
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
